In [4]:
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import cv2
from google.cloud import storage
from tensorflow.keras.utils import image_dataset_from_directory


2023-03-11 16:45:59.507216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
directory = '/content/drive/My Drive/Colab Notebooks/raw_data'
training_directory = f'{directory}/Training'
testing_directory = f'{directory}/Testing'

In [4]:
train_ds, val_ds = image_dataset_from_directory(
  '/Users/victorvoneisenhart-rothe/code/Victorvone/braintumorclassification/braintumorclassification/raw_data/Training/',
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255, 255),
  validation_split = 0.2,
  subset="both",
  color_mode = 'rgb',
  batch_size= 64)

test_ds = image_dataset_from_directory(
  '/Users/victorvoneisenhart-rothe/code/Victorvone/braintumorclassification/braintumorclassification/raw_data/Training/',
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255, 255),
  color_mode = 'rgb',
  batch_size= 64)

Found 7681 files belonging to 4 classes.
Using 6145 files for training.
Using 1536 files for validation.


2023-03-11 16:47:19.244063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 7681 files belonging to 4 classes.


In [10]:
train_ds.__len__()

<tf.Tensor: shape=(), dtype=int64, numpy=97>

In [32]:
from tensorflow.keras.applications import EfficientNetV2B3
from tensorflow.keras import layers, Sequential


In [33]:

base_model = EfficientNetV2B3(include_top=False,
                       weights='imagenet',
                       input_shape=(255,255,3),
                       pooling='max',
                       include_preprocessing=True) 

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
# model.add(layers.Dropout(0.3))
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.3))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(4, activation='softmax'))

base_model.trainable = False

model.summary()

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

52606240/52606240 [==============================] - 0s 0us/step
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b3 (Function  (None, 1536)             12930622  
 al)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1536)              0         
                                                                 
 dense_9 (Dense)             (None, 64)                98368     
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 4)                 260       
                                                                 
Total params: 13,029,250
Trainable params: 98,628
Non-t

In [34]:
from tensorflow.keras import callbacks

es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

model.fit(train_ds,
          validation_data=val_ds,
          batch_size=64, 
          epochs=100,
          callbacks=[es],
          verbose=1)

Epoch 1/100
36/36 [==============================] - 33s 488ms/step - loss: 2.0508 - accuracy: 0.3746 - val_loss: 1.0921 - val_accuracy: 0.5279
Epoch 2/100
36/36 [==============================] - 13s 317ms/step - loss: 1.1222 - accuracy: 0.4839 - val_loss: 0.8809 - val_accuracy: 0.6307
Epoch 3/100
36/36 [==============================] - 15s 394ms/step - loss: 0.9600 - accuracy: 0.5828 - val_loss: 0.7410 - val_accuracy: 0.7178
Epoch 4/100
36/36 [==============================] - 15s 381ms/step - loss: 0.8852 - accuracy: 0.6106 - val_loss: 0.6858 - val_accuracy: 0.7334
Epoch 5/100
36/36 [==============================] - 13s 314ms/step - loss: 0.8293 - accuracy: 0.6341 - val_loss: 0.5794 - val_accuracy: 0.7770
Epoch 6/100
36/36 [==============================] - 13s 320ms/step - loss: 0.7444 - accuracy: 0.6847 - val_loss: 0.5095 - val_accuracy: 0.8031
Epoch 7/100
36/36 [==============================] - 13s 320ms/step - loss: 0.6855 - accuracy: 0.6969 - val_loss: 0.4853 - val_accuracy:

In [35]:
model.evaluate(test_ds)

7/7 [==============================] - 3s 266ms/step - loss: 0.8453 - accuracy: 0.7208


[0.8452864289283752, 0.720812201499939]